## IMPORTING PACKAGES:

In [1]:
import math
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
import scipy
from scipy import stats
from scipy.stats import chi2_contingency
import os

##                                         BUSINESS PROBLEM-1

## BACKGROUND: 
The Lending Club is a peer-to-peer lending site where members make loans to each other. The site makes anonymized data on loans and borrowers publicly available.

## BUSINESS PROBLEM:
Using lending club loans data, the team would like to test below hypothesis on how different factors effecing each other

## DATA AVAILABLE:

In [2]:
stat=%cd "C:\Users\itsfe\Downloads\Basic Statistics - Hypothesis Testing"
loan=pd.read_csv('LoansData.csv')

C:\Users\itsfe\Downloads\Basic Statistics - Hypothesis Testing


## DATA CLEANING

In [3]:
loan.columns=['Amount_Requested','Amount_Funded_By_Investors','Interest_Rate','Loan_Length','Loan_Purpose','Debt_To_Income_Ratio','State','Home_Ownership','Monthly_Income','FICO_Range','Open_CREDIT_Lines','Revolving_CREDIT_Balance','Inquiries_in_the_Last_6 Months','Employment_Length']

loan['FICO_Range']=loan['FICO_Range'].str[-3:]
loan['FICO_Range']=pd.to_numeric(loan['FICO_Range'],errors='coerce')

loan['Employment_Length']=loan.Employment_Length.str.replace(r'\D+','')
loan['Employment_Length']=pd.to_numeric(loan['Employment_Length'],errors='coerce')

loan['Loan_Length']=loan.Loan_Length.apply(lambda x:float(str(x).replace('months','')))

loan['Interest_Rate']=loan['Interest_Rate'].str.replace('%','')
loan['Interest_Rate']=pd.to_numeric(loan['Interest_Rate'],errors='coerce')

loan['Debt_To_Income_Ratio']=loan['Debt_To_Income_Ratio'].str.replace('%','')
loan['Debt_To_Income_Ratio']=pd.to_numeric(loan['Debt_To_Income_Ratio'],errors='coerce')
loan.head(2)
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Amount_Requested                2499 non-null   float64
 1   Amount_Funded_By_Investors      2499 non-null   float64
 2   Interest_Rate                   2498 non-null   float64
 3   Loan_Length                     2500 non-null   float64
 4   Loan_Purpose                    2500 non-null   object 
 5   Debt_To_Income_Ratio            2499 non-null   float64
 6   State                           2500 non-null   object 
 7   Home_Ownership                  2499 non-null   object 
 8   Monthly_Income                  2499 non-null   float64
 9   FICO_Range                      2498 non-null   float64
 10  Open_CREDIT_Lines               2497 non-null   float64
 11  Revolving_CREDIT_Balance        2497 non-null   float64
 12  Inquiries_in_the_Last_6 Months  24

C:\Users\itsfe\AppData\Local\Temp\ipykernel_15948\3923857273.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  loan['Employment_Length']=loan.Employment_Length.str.replace(r'\D+','')


Treating Missing Values:

In [4]:
def miss_treat(x):
    if x.dtype=='float':
        x=x.fillna(x.median())
    else:
        x=x.fillna(x.mode()[0])
    return x

In [5]:
loan=loan.apply(miss_treat)
loan.head(10)


,Amount_Requested,Amount_Funded_By_Investors,Interest_Rate,Loan_Length,Loan_Purpose,Debt_To_Income_Ratio,State,Home_Ownership,Monthly_Income,FICO_Range,Open_CREDIT_Lines,Revolving_CREDIT_Balance,Inquiries_in_the_Last_6 Months,Employment_Length
0,20000.0,20000.0,8.90,36.0,debt_consolidation,14.90,SC,MORTGAGE,6541.67,739.0,14.0,14272.0,2.0,1.0
1,19200.0,19200.0,12.12,36.0,debt_consolidation,28.36,TX,MORTGAGE,4583.33,719.0,12.0,11140.0,1.0,2.0
2,35000.0,35000.0,21.98,60.0,debt_consolidation,23.81,CA,MORTGAGE,11500.00,694.0,14.0,21977.0,1.0,2.0
3,10000.0,9975.0,9.99,36.0,debt_consolidation,14.30,KS,MORTGAGE,3833.33,699.0,10.0,9346.0,0.0,5.0
4,12000.0,12000.0,11.71,36.0,credit_card,18.78,NJ,RENT,3195.00,699.0,11.0,14469.0,0.0,9.0
5,6000.0,6000.0,15.31,36.0,other,20.05,CT,OWN,4891.67,674.0,17.0,10391.0,2.0,3.0
6,10000.0,10000.0,7.90,36.0,debt_consolidation,26.09,MA,RENT,2916.67,724.0,10.0,15957.0,0.0,10.0
7,33500.0,33450.0,17.14,60.0,credit_card,14.70,LA,MORTGAGE,13863.42,709.0,12.0,27874.0,0.0,10.0
8,14675.0,14675.0,14.33,36.0,credit_card,26.92,CA,RENT,3150.00,689.0,9.0,7246.0,1.0,8.0
9,7000.0,7000.0,6.91,36.0,credit_card,7.10,CA,RENT,5000.00,719.0,8.0,7612.0,0.0,3.0


## a. Intrest rate is varied for different loan amounts (Less intrest charged for high loan amounts)

## Step 1

## Defining Parameters

Null Hypothesis,H0 : There is no significant relationship between Interest rate and Loan Amounts

Alternative Hypothesis,Ha : There is significant relationship between Interest rate and Loan Amounts

## Step 2

## Correlation Test

In [6]:
rvalue,pval=stats.pearsonr(loan.Interest_Rate,loan.Amount_Funded_By_Investors)
print(f'rvalue is:{rvalue}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('There is a significant relationship between Interest rate and Loan Amounts')
else:
    print('There is no significant relationship between Interest rate and Loan Amounts')

rvalue is:0.3374589686089662
pvalue is:1.2376653981362837e-67
There is a significant relationship between Interest rate and Loan Amounts


## Step3

## Conclusion

Since pvalue is less than 0.05 and r value is positive ,we can reject H0 and accept Ha,ie, Interest Rate varies directly with different Loan Amounts.

## b. Loan length is directly effecting intrest rate.

## Step 1

## Defining Parameters
Null Hypothesis,H0 : Loan length is not effecting intrest rate.

Alternative Hypothesis,Ha :Loan length is directly effecting intrest rate.

## Step 2

## Correlation Test

In [7]:
rvalue,pval=stats.pearsonr(loan.Loan_Length,loan.Interest_Rate)
print(f'rvalue is:{rvalue}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('There is a significant relationship between Loan Length and Interest Rate')
else:
    print('There is no significant relationship between Loan Length and Interest Rate')


rvalue is:0.42416737515954295
pvalue is:9.605991607854548e-110
There is a significant relationship between Loan Length and Interest Rate


## Step 3

## Conclusion

Since pvalue is less than 0.05 and r value is positive ,we can reject H0 and accept Ha, ie,  Loan Length is directly affecting Interest Rate

## c. Interest rate varies for different purpose of loans.

## Step 1

## Defining Parameters
Null Hypothesis,H0 : Interest rate does not vary with loan purpose.

Alternative Hypothesis,Ha :Interest rate varies with loan purpose.

## Step 2

## Ftest

In [8]:
dt_con=loan.Interest_Rate[loan.Loan_Purpose=='debt_consolidation']
cr_cd=loan.Interest_Rate[loan.Loan_Purpose=='credit_card']
oth=loan.Interest_Rate[loan.Loan_Purpose=='other']       
mov=loan.Interest_Rate[loan.Loan_Purpose=='moving']
car=loan.Interest_Rate[loan.Loan_Purpose=='car']
vac=loan.Interest_Rate[loan.Loan_Purpose=='vacation']
home=loan.Interest_Rate[loan.Loan_Purpose=='home_improvement']
house=loan.Interest_Rate[loan.Loan_Purpose=='house']
maj_pur=loan.Interest_Rate[loan.Loan_Purpose=='major_purchase']
edu=loan.Interest_Rate[loan.Loan_Purpose=='educational']
med=loan.Interest_Rate[loan.Loan_Purpose=='medical']
wed=loan.Interest_Rate[loan.Loan_Purpose=='wedding']
bus=loan.Interest_Rate[loan.Loan_Purpose=='small_business']

In [9]:
fval,pval=stats.f_oneway(dt_con,cr_cd,oth,mov,car,vac,home,house,maj_pur,edu,med,wed,bus)
print(f'fvalue is:{fval}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('There is a significant relationship between Loan Purpose and Interest Rate')
else:
    print('There is no significant relationship between Loan Purpose and Interest Rate')


fvalue is:7.901182879604524
pvalue is:1.1636926239581812e-14
There is a significant relationship between Loan Purpose and Interest Rate


## Step 3

## Conclusion

Since pvalue is less than 0.05 ,we can reject H0 and accept Ha, ie, Interest Rate varies for Loan Purpose.

## d. There is relationship between FICO scores and Home Ownership. It means that, People with owning home will have high FICO scores

## Step 1

## Defining Parameters
Null Hypothesis,H0 : There is no significant relationship between FICO Scores and Home Ownership.

Alternative Hypothesis,Ha :There is a significant relationship between FICO Scores and Home Ownership.

## Step 2

## Ftest

In [10]:
mort=loan.FICO_Range[loan.Home_Ownership=='MORTGAGE']
own=loan.FICO_Range[loan.Home_Ownership=='OWN']
rent=loan.FICO_Range[loan.Home_Ownership=='RENT']

In [11]:
fval,pval=stats.f_oneway(mort,own,rent)
print(f'fvalue is:{fval}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('There is a significant relationship between FICO Scores and Home Ownership')
else:
    print('There is no significant relationship between FICO Scores and Home Ownership')

fvalue is:32.5186475595199
pvalue is:1.1443189380841834e-14
There is a significant relationship between FICO Scores and Home Ownership


## Step 3

## Conclusion

Since pvalue is less than 0.05 ,we can reject H0 and accept Ha, ie, There is a significant relationship between FICO Scores and Home Ownership. People owning home have high FICO scores.

## BUSINESS PROBLEM - 2

We would like to assess if there is any difference in the average price quotes provided by Mary and Barry.

## DATA AVAILABLE:

In [12]:
price=pd.read_csv('Price_Quotes.csv')
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Order_Number  12 non-null     int64
 1   Barry_Price   12 non-null     int64
 2   Mary_Price    12 non-null     int64
dtypes: int64(3)
memory usage: 416.0 bytes


## Step 1

## Defining Parameters
Null Hypothesis,H0 : There is significant difference in the average price quotes provided by Mary and Barry.

Alternative Hypothesis,Ha :There is no significant difference in the average price quotes provided by Mary and Barry.

## Step 2

## Ttest Independent

In [13]:
tval,pval=stats.ttest_ind(price['Barry_Price'],price['Mary_Price'])
print(f'tvalue is:{tval}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('There is a significant relationship between Barry Price and Mary Price')
else:
    print('There is no significant relationship between Barry Price and Mary Price')

tvalue is:1.4147436739281787
pvalue is:0.17114226132118285
There is no significant relationship between Barry Price and Mary Price


## Step 3

## Conclusion

Since pvalue is greater than 0.05 ,we cannot reject H0 , ie, there is difference in the average price quotes provided by Mary and Barry.

## BUSINESS PROBLEM-3:

## Determine what effect, if any, the reengineering effort had on the incidence behavioral problems and staff turnover. i.e To determine if the reengineering effort changed the critical incidence rate. Is there evidence that the critical incidence rate improved?


## DATA AVAILABLE:

In [14]:
treat_fac=pd.read_csv('Treatment_Facility.csv')
treat_fac.rename(columns={'VAR4':'TRFF','VAR5':'CI'},inplace=True)

In [15]:
treat_fac

,Month,Reengineer,Employee_Turnover,TRFF,CI
0,1,Prior,0.0000,24.390244,42.682927
1,2,Prior,6.0606,19.354839,25.806452
2,3,Prior,12.1212,35.087719,146.198830
3,4,Prior,3.3333,18.404908,110.429448
4,5,Prior,12.9032,17.964072,23.952096
5,6,Prior,9.6774,41.176471,47.058824
6,7,Prior,11.7647,13.422819,0.000000
7,8,Prior,11.4286,31.250000,25.000000
8,9,Prior,23.0769,17.241379,132.183908
9,10,Prior,15.0000,16.574586,16.574586


## Step 1

## Defining Parameters
Null Hypothesis,H0 : There is no effect of Reengineering on Critical Incidence Rate.

Alternative Hypothesis,Ha :There is an effect of Reengineering on Critical Incidence Rate.

## Step 2

## FTest

In [16]:
prior=treat_fac.CI[treat_fac.Reengineer=='Prior']
post=treat_fac.CI[treat_fac.Reengineer=='Post']

In [17]:
fval,pval=stats.f_oneway(prior,post)
print(f'fvalue is:{fval}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('There is an effect of Reengineering on Critical Incidence Rate')
else:
    print('There is no effect of Reengineering on Critical Incidence Rate')

fvalue is:2.650105376271948
pvalue is:0.12091989189884142
There is no effect of Reengineering on Critical Incidence Rate


## Step 3

## Conclusion

Since pvalue is greater than 0.05 ,we cannot reject H0 , ie, there is no effect of Reengineering on Critical Incidence Rate(CI). The CI could not be improved by the Reengineering efforts.

## BUSINESS PROBLEM-4

BUSINESS PROBLEM: We will focus on the prioritization system. If the system is working, then
high priority jobs, on average, should be completed more quickly than medium priority jobs,
and medium priority jobs should be completed more quickly than low priority jobs. Use the
data provided to determine whether this is, in fact, occurring.

## DATA AVAILABLE:

In [18]:
prio=pd.read_csv('Priority_Assessment.csv')
prio

,Days,Priority
0,3.3,High
1,7.9,Medium
2,0.3,High
3,0.7,Medium
4,8.6,Medium
...,...,...
637,2.5,Low
638,0.3,High
639,0.3,Medium
640,1.3,Medium


## Step 1

## Defining Parameters
Null Hypothesis,H0 : Work is not completed as per the priority scale.

Alternative Hypothesis,Ha :Work is completed as per the priority scale.

## Step 2

## Anova Test

In [19]:
high=prio.Days[prio.Priority=='High']
med=prio.Days[prio.Priority=='Medium']
low=prio.Days[prio.Priority=='Low']

In [20]:
fval,pval=stats.f_oneway(high,med,low)

In [21]:
print(f'fvalue is:{fval}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('Work is completed as per the priority scale')
else:
    print('Work is not completed as per the priority scale')

fvalue is:1.812311010076072
pvalue is:0.16411459461716182
Work is not completed as per the priority scale


## Step 3

## Conclusion

Since pvalue is greater than 0.05 ,we can reject H0 ,ie, work is not completed as per the priority scale.The prioritization system is not working.

## BUSINESS PROBLEM-5

## Use the survey results to address the following questions

## Data Available

In [49]:
films=pd.read_csv('Films.csv')
films.head()


,_rowstate_,Movie,Gender,Marital_Status,Sinage,Parking,Clean,Overall,Age,Income,Hear_About
123,0,Ferris Buellers Day Off,2,1,1.0,1.0,1.0,1.0,NaN,2.0,NaN
171,0,Willy Wonka,2,2,2.0,2.0,1.0,1.0,NaN,2.0,"3,4"


## Data Cleaning

In [23]:
films=films.drop(columns='_rowstate_',axis=0)

In [50]:
def miss_treat(x):
    if x.dtype=='float' :
        x=x.fillna(x.median())
    else:
        x=x.fillna(x.mode()[0])
    return x

In [53]:
films=films.apply(miss_treat)
films.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   _rowstate_      330 non-null    int64  
 1   Movie           330 non-null    object 
 2   Gender          330 non-null    object 
 3   Marital_Status  330 non-null    object 
 4   Sinage          330 non-null    float64
 5   Parking         330 non-null    float64
 6   Clean           330 non-null    float64
 7   Overall         330 non-null    float64
 8   Age             330 non-null    float64
 9   Income          330 non-null    float64
 10  Hear_About      330 non-null    object 
dtypes: float64(6), int64(1), object(4)
memory usage: 28.5+ KB


## What is the overall level of customer satisfaction?

In [26]:
mean=math.ceil(films.Overall.mean())
if mean==1:
    print('Overall level of customer satisfaction is: Excellent')
elif mean==2:
    print('Overall level of customer satisfaction is: Good')
elif mean==3:
    print('Overall level of customer satisfaction is: Average')
elif mean==4:
    print('Overall level of customer satisfaction is: Poor')
elif mean==5:
    print('Overall level of customer satisfaction is: Very poor')

Overall level of customer satisfaction is: Good


## What factors are linked to satisfaction?

## Checking factors' relation with Satisfaction

## 1. Sinage Facilities

## Correlation Test

In [27]:
rvalue,pval=stats.pearsonr(films.Sinage,films.Overall)
print(f'rvalue is:{rvalue}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('Satisfaction varies with sinage maintenance')
else:
    print('Satisfaction does not vary with sinage maintenance ')

rvalue is:0.38210280422395665
pvalue is:6.501065434766499e-13
Satisfaction varies with sinage maintenance


## 2. Parking Facilities

In [28]:
rvalue,pval=stats.pearsonr(films.Parking,films.Overall)
print(f'rvalue is:{rvalue}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('Satisfaction varies with parking maintenance')
else:
    print('Satisfaction does not vary with parking maintenance ')

rvalue is:0.5144289487855273
pvalue is:1.0840276087789278e-23
Satisfaction varies with parking maintenance


## 3. Cleanliness

In [29]:
rvalue,pval=stats.pearsonr(films.Clean,films.Overall)
print(f'rvalue is:{rvalue}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('Satisfaction varies with cleanliness')
else:
    print('Satisfaction does not vary with cleanliness ')

rvalue is:0.3508222789070923
pvalue is:5.437550007439627e-11
Satisfaction varies with cleanliness


## What is the demographic profile of Film on the Rocks patrons?

## Demographic Profile

## Checking Relation with Gender using Chi-Square Test

In [38]:
gender=pd.crosstab(films.Gender,films.Movie)

In [63]:
chi_val, p_val, dof, expected=chi2_contingency(gender)

In [64]:
print(f'chivalue is:{chi_val}')
print(f'pvalue is:{p_val}')
if p_val<0.05:
    print('Gender affects the film preference ')
else:
    print('Gender does not affect the film preference')

chivalue is:50.80551289003373
pvalue is:3.2405158090240795e-09
Gender affects the film preference 


## Checking Relation with Age Using Anova Test

In [57]:
old_school=films.Age[films.Movie=='Old School']
willy_wonka=films.Age[films.Movie=='Willy Wonka']
ferris=films.Age[films.Movie=="Ferris Buellers Day Off"]

In [58]:
fval,pval=stats.f_oneway(old_school,willy_wonka,ferris)

In [60]:
print(f'fvalue is:{fval}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('Preference for all films are affected by age ')
else:
    print('At least one of the films is not affected by age')

fvalue is:0.40397339240723484
pvalue is:0.6679946281373063
At least one of the films is not affected by age


## Checking Relation with Marital Status Using Chisquare Test

In [62]:
mar_stats=pd.crosstab(films.Marital_Status,films.Movie)

In [65]:
chi_val, p_val, dof, expected=chi2_contingency(mar_stats)

In [66]:
print(f'chivalue is:{chi_val}')
print(f'pvalue is:{p_val}')
if pval<0.05:
    print('Marital Status affects the film preference ')
else:
    print('Marital Status does not affect the film preference')

chivalue is:50.48412336907947
pvalue is:3.298300328839479e-08
Marital Status does not affect the film preference


## Checking Relation with Income Using Anova Test

In [67]:
old_school=films.Income[films.Movie=='Old School']
willy_wonka=films.Income[films.Movie=='Willy Wonka']
ferris=films.Income[films.Movie=="Ferris Buellers Day Off"]

In [68]:
fval,pval=stats.f_oneway(old_school,willy_wonka,ferris)

In [69]:
print(f'fvalue is:{fval}')
print(f'pvalue is:{pval}')
if pval<0.05:
    print('Preference for all films are affected by income')
else:
    print('At least one of the films is not affected by income')

fvalue is:0.40796312620752384
pvalue is:0.6653413917049096
At least one of the films is not affected by income


## In what media outlet(s) should the film series be advertised?

In [ ]:
mode=films.Hear_About.mode()[0]
mode

In [ ]:
if mode==1:
    print('Film Series should be advertised more in Television')
elif mode==2:
    print('Film Series should be advertised more in Newspaper')
elif mode==3:
    print('Film Series should be advertised more in Radio')
elif mode==4:
    print('Film Series should be advertised more in Website')
else:
    print('Film Series should be advertised more by word-of-mouth')